In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

sm= SMOTE(random_state= 15)
tl= TomekLinks()

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
from sklearn.feature_selection import SelectKBest, f_classif

selector= SelectKBest(score_func= f_classif, k= 10)
X_sm_tl_selected= selector.fit_transform(X_sm_tl, y_sm_tl)
X_sm_tl_test_selected= selector.transform(X_test)

In [8]:
feature_names= X.columns
selected_indices= selector.get_support(indices=True)
selected_features= feature_names[selected_indices]
print("Selected Features: ", selected_features)

Selected Features:  Index(['Cholesterol', 'Family History', 'Obesity', 'Alcohol Consumption',
       'Diet', 'Sedentary Hours Per Day', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'HBP'],
      dtype='object')


--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state=15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62      1125
           1       0.34      0.36      0.35       628

    accuracy                           0.52      1753
   macro avg       0.48      0.48      0.48      1753
weighted avg       0.53      0.52      0.52      1753

Precision: 0.33839150227617604
Accuracy: 0.5202509982886481
AUC: 0.48345222929936316


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.64      0.64      1125
           1       0.37      0.38      0.38       628

    accuracy                           0.55      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.55      0.55      1753

Precision: 0.37111801242236025
Accuracy: 0.547062179121506
AUC: 0.5102866242038218


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.6425533143957446


In [12]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.65      0.90      0.75      1125
           1       0.39      0.11      0.18       628

    accuracy                           0.62      1753
   macro avg       0.52      0.51      0.46      1753
weighted avg       0.55      0.62      0.55      1753

Precision: 0.3879781420765027
Accuracy: 0.6183685111237878
AUC: 0.5180488322717622


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.63      0.55      0.59      1125
           1       0.35      0.43      0.38       628

    accuracy                           0.51      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.51      0.51      1753

Precision: 0.3462033462033462
Accuracy: 0.5054192812321734
AUC: 0.48507430997876855


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.6874087350540558


In [15]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.81      0.71      1125
           1       0.34      0.17      0.23       628

    accuracy                           0.58      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.58      0.54      1753

Precision: 0.33753943217665616
Accuracy: 0.583000570450656
AUC: 0.4967098372257608


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62      1125
           1       0.34      0.36      0.35       628

    accuracy                           0.52      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.52      0.52      1753

Precision: 0.3409090909090909
Accuracy: 0.5219623502567028
AUC: 0.47749044585987266


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.63      0.76      0.69      1125
           1       0.32      0.21      0.25       628

    accuracy                           0.56      1753
   macro avg       0.48      0.48      0.47      1753
weighted avg       0.52      0.56      0.53      1753

Precision: 0.3226600985221675
Accuracy: 0.5596120935539076
AUC: 0.48904600141542814


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.87      0.74      1125
           1       0.34      0.12      0.18       628

    accuracy                           0.60      1753
   macro avg       0.49      0.50      0.46      1753
weighted avg       0.53      0.60      0.54      1753

Precision: 0.3409090909090909
Accuracy: 0.6018254420992584
AUC: 0.506966737438075


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best Precision Score: 0.7684175281056522


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.64      0.90      0.75      1125
           1       0.33      0.09      0.14       628

    accuracy                           0.61      1753
   macro avg       0.48      0.49      0.44      1753
weighted avg       0.53      0.61      0.53      1753

Precision: 0.32926829268292684
Accuracy: 0.609811751283514
AUC: 0.5029370134465676


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.64      0.81      0.71      1125
           1       0.34      0.18      0.23       628

    accuracy                           0.58      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.58      0.54      1753

Precision: 0.34146341463414637
Accuracy: 0.5824301197946378
AUC: 0.4833319179051663


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.7778415767410364


In [25]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.41      0.03      0.05       628

    accuracy                           0.64      1753
   macro avg       0.53      0.50      0.41      1753
weighted avg       0.56      0.64      0.52      1753

Precision: 0.41025641025641024
Accuracy: 0.6377638334284085
AUC: 0.49078627034677985


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.33      0.01      0.03       628

    accuracy                           0.64      1753
   macro avg       0.49      0.50      0.40      1753
weighted avg       0.53      0.64      0.51      1753

Precision: 0.3333333333333333
Accuracy: 0.6366229321163719
AUC: 0.48384005661712665


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7397227355024073


In [28]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.96      0.77      1125
           1       0.40      0.05      0.09       628

    accuracy                           0.63      1753
   macro avg       0.52      0.50      0.43      1753
weighted avg       0.56      0.63      0.53      1753

Precision: 0.4
Accuracy: 0.6332002281802624
AUC: 0.4974578910120311


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.65      0.83      0.73      1125
           1       0.37      0.18      0.25       628

    accuracy                           0.60      1753
   macro avg       0.51      0.51      0.49      1753
weighted avg       0.55      0.60      0.55      1753

Precision: 0.3745928338762215
Accuracy: 0.5978322875071307
AUC: 0.4983177636234961


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.01}
Best Precision Score: 0.7768088685851844
              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.40      0.02      0.04       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.41      1753
weighted avg       0.56      0.64      0.51      1753

Precision: 0.4
Accuracy: 0.6377638334284085
AUC: 0.4935902335456475


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 4288, number of negative: 4499
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000422 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 8787, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487994 -> initscore=-0.048035
[LightGBM] [Info] Start training from score -0.048035
              precision    recall  f1-score   support

           0       0.64      0.91      0.75      1125
           1       0.34      0.09      0.14       628

    accuracy                           0.61      1753
   macro avg       0.49      0.50      0.44      1753
weighted avg       0.53      0.61      0.53      1753

Precision: 0.34394904458598724
Accuracy: 0.6138049058756417
AUC: 0.5028039631988677


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4499
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000497 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 8787, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487994 -> initscore=-0.048035
[LightGBM] [Info] Start training from score -0.048035
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.8387833246116143
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_sa

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.026059
0:	learn: 0.6894010	total: 150ms	remaining: 2m 29s
1:	learn: 0.6862085	total: 155ms	remaining: 1m 17s
2:	learn: 0.6822416	total: 161ms	remaining: 53.6s
3:	learn: 0.6785305	total: 166ms	remaining: 41.4s
4:	learn: 0.6759745	total: 171ms	remaining: 34s
5:	learn: 0.6708061	total: 178ms	remaining: 29.5s
6:	learn: 0.6654613	total: 185ms	remaining: 26.3s
7:	learn: 0.6615750	total: 191ms	remaining: 23.7s
8:	learn: 0.6591224	total: 196ms	remaining: 21.6s
9:	learn: 0.6564609	total: 200ms	remaining: 19.8s
10:	learn: 0.6531842	total: 209ms	remaining: 18.8s
11:	learn: 0.6504895	total: 213ms	remaining: 17.6s
12:	learn: 0.6465190	total: 220ms	remaining: 16.7s
13:	learn: 0.6444900	total: 226ms	remaining: 15.9s
14:	learn: 0.6414865	total: 231ms	remaining: 15.1s
15:	learn: 0.6390188	total: 236ms	remaining: 14.5s
16:	learn: 0.6362132	total: 240ms	remaining: 13.9s
17:	learn: 0.6341746	total: 245ms	remaining: 13.4s
18:	learn: 0.6326438	total: 250ms	remaining: 12.9s
19:	learn: 

176:	learn: 0.5303870	total: 1.04s	remaining: 4.84s
177:	learn: 0.5295262	total: 1.05s	remaining: 4.84s
178:	learn: 0.5292376	total: 1.05s	remaining: 4.84s
179:	learn: 0.5290673	total: 1.06s	remaining: 4.84s
180:	learn: 0.5288753	total: 1.07s	remaining: 4.85s
181:	learn: 0.5286781	total: 1.08s	remaining: 4.87s
182:	learn: 0.5285473	total: 1.09s	remaining: 4.88s
183:	learn: 0.5283822	total: 1.1s	remaining: 4.89s
184:	learn: 0.5280458	total: 1.11s	remaining: 4.88s
185:	learn: 0.5278287	total: 1.11s	remaining: 4.87s
186:	learn: 0.5276564	total: 1.12s	remaining: 4.86s
187:	learn: 0.5273112	total: 1.12s	remaining: 4.85s
188:	learn: 0.5272186	total: 1.13s	remaining: 4.84s
189:	learn: 0.5268869	total: 1.13s	remaining: 4.82s
190:	learn: 0.5266937	total: 1.14s	remaining: 4.81s
191:	learn: 0.5265453	total: 1.14s	remaining: 4.8s
192:	learn: 0.5262325	total: 1.15s	remaining: 4.79s
193:	learn: 0.5257982	total: 1.15s	remaining: 4.78s
194:	learn: 0.5256774	total: 1.16s	remaining: 4.77s
195:	learn: 0.

342:	learn: 0.4962306	total: 1.93s	remaining: 3.7s
343:	learn: 0.4960543	total: 1.94s	remaining: 3.7s
344:	learn: 0.4958606	total: 1.94s	remaining: 3.69s
345:	learn: 0.4957197	total: 1.95s	remaining: 3.68s
346:	learn: 0.4955218	total: 1.95s	remaining: 3.67s
347:	learn: 0.4952832	total: 1.96s	remaining: 3.67s
348:	learn: 0.4950357	total: 1.96s	remaining: 3.66s
349:	learn: 0.4949141	total: 1.97s	remaining: 3.66s
350:	learn: 0.4945037	total: 1.97s	remaining: 3.65s
351:	learn: 0.4942926	total: 1.98s	remaining: 3.64s
352:	learn: 0.4939652	total: 1.98s	remaining: 3.64s
353:	learn: 0.4937864	total: 1.99s	remaining: 3.63s
354:	learn: 0.4935820	total: 1.99s	remaining: 3.62s
355:	learn: 0.4934437	total: 2s	remaining: 3.62s
356:	learn: 0.4932687	total: 2s	remaining: 3.61s
357:	learn: 0.4931075	total: 2.01s	remaining: 3.6s
358:	learn: 0.4929876	total: 2.01s	remaining: 3.59s
359:	learn: 0.4928604	total: 2.02s	remaining: 3.58s
360:	learn: 0.4926461	total: 2.02s	remaining: 3.58s
361:	learn: 0.4923792

525:	learn: 0.4635433	total: 2.83s	remaining: 2.55s
526:	learn: 0.4633510	total: 2.84s	remaining: 2.55s
527:	learn: 0.4632305	total: 2.84s	remaining: 2.54s
528:	learn: 0.4630948	total: 2.85s	remaining: 2.54s
529:	learn: 0.4629766	total: 2.85s	remaining: 2.53s
530:	learn: 0.4627410	total: 2.86s	remaining: 2.53s
531:	learn: 0.4626383	total: 2.87s	remaining: 2.52s
532:	learn: 0.4625367	total: 2.88s	remaining: 2.52s
533:	learn: 0.4623837	total: 2.88s	remaining: 2.51s
534:	learn: 0.4622512	total: 2.88s	remaining: 2.51s
535:	learn: 0.4620226	total: 2.89s	remaining: 2.5s
536:	learn: 0.4619695	total: 2.9s	remaining: 2.5s
537:	learn: 0.4618154	total: 2.9s	remaining: 2.49s
538:	learn: 0.4617572	total: 2.9s	remaining: 2.48s
539:	learn: 0.4615874	total: 2.91s	remaining: 2.48s
540:	learn: 0.4613851	total: 2.92s	remaining: 2.47s
541:	learn: 0.4610931	total: 2.92s	remaining: 2.47s
542:	learn: 0.4609391	total: 2.92s	remaining: 2.46s
543:	learn: 0.4608463	total: 2.93s	remaining: 2.45s
544:	learn: 0.460

701:	learn: 0.4396958	total: 3.72s	remaining: 1.58s
702:	learn: 0.4396022	total: 3.73s	remaining: 1.57s
703:	learn: 0.4394846	total: 3.73s	remaining: 1.57s
704:	learn: 0.4393817	total: 3.74s	remaining: 1.56s
705:	learn: 0.4392378	total: 3.74s	remaining: 1.56s
706:	learn: 0.4390844	total: 3.75s	remaining: 1.55s
707:	learn: 0.4389753	total: 3.76s	remaining: 1.55s
708:	learn: 0.4387997	total: 3.76s	remaining: 1.54s
709:	learn: 0.4385770	total: 3.77s	remaining: 1.54s
710:	learn: 0.4384581	total: 3.77s	remaining: 1.53s
711:	learn: 0.4383288	total: 3.78s	remaining: 1.53s
712:	learn: 0.4382341	total: 3.79s	remaining: 1.52s
713:	learn: 0.4381213	total: 3.79s	remaining: 1.52s
714:	learn: 0.4380130	total: 3.8s	remaining: 1.51s
715:	learn: 0.4378643	total: 3.8s	remaining: 1.51s
716:	learn: 0.4377437	total: 3.81s	remaining: 1.5s
717:	learn: 0.4376429	total: 3.81s	remaining: 1.5s
718:	learn: 0.4374824	total: 3.82s	remaining: 1.49s
719:	learn: 0.4373832	total: 3.82s	remaining: 1.49s
720:	learn: 0.43

868:	learn: 0.4205555	total: 4.61s	remaining: 695ms
869:	learn: 0.4205006	total: 4.62s	remaining: 690ms
870:	learn: 0.4204073	total: 4.62s	remaining: 685ms
871:	learn: 0.4202846	total: 4.63s	remaining: 679ms
872:	learn: 0.4201844	total: 4.63s	remaining: 674ms
873:	learn: 0.4200524	total: 4.64s	remaining: 668ms
874:	learn: 0.4199046	total: 4.64s	remaining: 663ms
875:	learn: 0.4197909	total: 4.65s	remaining: 658ms
876:	learn: 0.4196595	total: 4.65s	remaining: 652ms
877:	learn: 0.4195578	total: 4.66s	remaining: 647ms
878:	learn: 0.4194790	total: 4.66s	remaining: 642ms
879:	learn: 0.4193472	total: 4.67s	remaining: 636ms
880:	learn: 0.4192323	total: 4.67s	remaining: 631ms
881:	learn: 0.4191138	total: 4.67s	remaining: 625ms
882:	learn: 0.4190152	total: 4.68s	remaining: 620ms
883:	learn: 0.4189494	total: 4.68s	remaining: 615ms
884:	learn: 0.4188611	total: 4.69s	remaining: 609ms
885:	learn: 0.4187470	total: 4.69s	remaining: 604ms
886:	learn: 0.4186156	total: 4.7s	remaining: 599ms
887:	learn: 0

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6862124	total: 33.8ms	remaining: 1.66s
1:	learn: 0.6773404	total: 66.1ms	remaining: 1.59s
2:	learn: 0.6701047	total: 101ms	remaining: 1.58s
3:	learn: 0.6659017	total: 131ms	remaining: 1.5s
4:	learn: 0.6576022	total: 149ms	remaining: 1.34s
5:	learn: 0.6527330	total: 178ms	remaining: 1.3s
6:	learn: 0.6481070	total: 213ms	remaining: 1.3s
7:	learn: 0.6453361	total: 252ms	remaining: 1.32s
8:	learn: 0.6408455	total: 287ms	remaining: 1.31s
9:	learn: 0.6346808	total: 320ms	remaining: 1.28s
10:	learn: 0.6303236	total: 352ms	remaining: 1.25s
11:	learn: 0.6272889	total: 390ms	remaining: 1.23s
12:	learn: 0.6237045	total: 425ms	remaining: 1.21s
13:	learn: 0.6196548	total: 462ms	remaining: 1.19s
14:	learn: 0.6162805	total: 497ms	remaining: 1.16s
15:	learn: 0.6138869	total: 531ms	remaining: 1.13s
16:	learn: 0.6107133	total: 567ms	remaining: 1.1s
17:	learn: 0.6081842	total: 608ms	remaining: 1.08s
18:	learn: 0.6057583	total: 641m

--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
10              Tuned AdaBoost Precision:         0.410256
12      Tuned GradientBoosting Precision:         0.400000
14                   Tuned XGB Precision:         0.400000
2          Tuned Decision Tree Precision:         0.387978
13                         XGB Precision:         0.374593
1                Decision Tree Precision:         0.371118
17                    CatBoost Precision:         0.352941
3         KNeighborsClassifier Precision:         0.346203
15                        LGBM Precision:         0.343949
9                     AdaBoost Precision:         0.341463
5                   GaussianNB Precision:         0.340909
7                Random Forest Precision:         0.340909
0          Logistic Regression Precision:         0.338392
4   Tuned KNeighborsClassifier Precision:         0.337539
11            GradientBoosting Precision:         0.333333
8          Tuned Random Forest Precision:         0.3292

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
14                   Tuned XGB Accuracy:        0.637764
10              Tuned AdaBoost Accuracy:        0.637764
16                  Tuned LGBM Accuracy:        0.636623
11            GradientBoosting Accuracy:        0.636623
12      Tuned GradientBoosting Accuracy:        0.633200
17                    CatBoost Accuracy:        0.621791
2          Tuned Decision Tree Accuracy:        0.618369
15                        LGBM Accuracy:        0.613805
18              Tuned CatBoost Accuracy:        0.611523
8          Tuned Random Forest Accuracy:        0.609812
7                Random Forest Accuracy:        0.601825
13                         XGB Accuracy:        0.597832
4   Tuned KNeighborsClassifier Accuracy:        0.583001
9                     AdaBoost Accuracy:        0.582430
6                          SVM Accuracy:        0.559612
1                Decision Tree Accuracy:        0.547062
5                   GaussianNB 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
2          Tuned Decision Tree AUC:   0.518049
1                Decision Tree AUC:   0.510287
17                    CatBoost AUC:   0.508125
7                Random Forest AUC:   0.506967
8          Tuned Random Forest AUC:   0.502937
15                        LGBM AUC:   0.502804
13                         XGB AUC:   0.498318
12      Tuned GradientBoosting AUC:   0.497458
4   Tuned KNeighborsClassifier AUC:   0.496710
14                   Tuned XGB AUC:   0.493590
10              Tuned AdaBoost AUC:   0.490786
16                  Tuned LGBM AUC:   0.490050
6                          SVM AUC:   0.489046
3         KNeighborsClassifier AUC:   0.485074
11            GradientBoosting AUC:   0.483840
0          Logistic Regression AUC:   0.483452
9                     AdaBoost AUC:   0.483332
5                   GaussianNB AUC:   0.477490
18              Tuned CatBoost AUC:   0.476539
